### LSTM

In [ ]:
# importer
import yfinance as yfin
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from torch.nn import Module, Linear
from torch.optim import Adam
from ml_utils import parse_config, vizualization